#2do Parcial - IA - 4 en raya

Alumno: Baldiviezo Cruz Dereck Fernando

Aplicación:

Cuatro en raya
Con el objetivo de ilustrar el aprendizaje por refuerzo (AxR) en un entorno más complejo, aplicaremos esta técnica al juego de Cuatro en raya. Este juego se juega en un tablero de 6 filas por 7 columnas. Dos jugadores se turnan para soltar una ficha en una de las columnas, y la ficha ocupa la posición más baja disponible. Gana el jugador que logre alinear cuatro fichas consecutivas en línea horizontal, vertical o diagonal.

Representación del entorno y del estado
Cada estado del juego se representa como una matriz de 6x7. Las posiciones vacías se codifican como 0, las fichas del jugador 1 como 1, y las del jugador 2 como -1. A partir de esta codificación, podemos representar un estado como una cadena de 42 números (flattened).

Función de valor
Como en el ejemplo de "Tres en raya", se utilizará una función de valor que asigna a cada estado una estimación de su probabilidad de victoria. Inicialmente, los estados no terminales tendrán valor 0.5. Los estados en los que se gana tendrán un valor de 1 y los de pérdida un valor de 0.

Actualización de la función de valor
Utilizaremos la misma fórmula de actualización temporal-diferencial (TD):

V(St) ← V(St) + α [V(St+1) − V(St)]

In [12]:
import numpy as np
from tqdm import tqdm
ROWS = 6
COLS = 7

class Board:
    def __init__(self):
        self.state = np.zeros((ROWS, COLS))

    def valid_moves(self):
        return [c for c in range(COLS) if self.state[0, c] == 0]

    def update(self, symbol, col):
        for row in reversed(range(ROWS)):
            if self.state[row, col] == 0:
                self.state[row, col] = symbol
                return row, col
        raise ValueError("Columna llena!")

    def is_game_over(self):
        for r in range(ROWS):
            for c in range(COLS):
                if self.state[r, c] == 0:
                    continue
                s = self.state[r, c]
                # Horizontal
                if c <= COLS - 4 and all(self.state[r, c+i] == s for i in range(4)):
                    return s
                # Vertical
                if r <= ROWS - 4 and all(self.state[r+i, c] == s for i in range(4)):
                    return s
                # Diagonal /
                if r >= 3 and c <= COLS - 4 and all(self.state[r-i, c+i] == s for i in range(4)):
                    return s
                # Diagonal \
                if r <= ROWS - 4 and c <= COLS - 4 and all(self.state[r+i, c+i] == s for i in range(4)):
                    return s
        if len(self.valid_moves()) == 0:
            return 0
        return None

    def reset(self):
        self.state = np.zeros((ROWS, COLS))


#Clase Agent:


In [13]:
class Agent():
    def __init__(self, alpha=0.5, prob_exp=0.1):
        self.value_function = {}
        self.alpha = alpha
        self.prob_exp = prob_exp
        self.positions = []

    def reset(self):
        self.positions = []

    def move(self, board, explore=True):
        valid_moves = board.valid_moves()
        # Exploración
        if explore and np.random.rand() < self.prob_exp:
            col = np.random.choice(valid_moves)
            return col
        # Explotación
        max_value = -1e9
        best_col = None
        for col in valid_moves:
            temp_board = board.state.copy()
            for row in reversed(range(ROWS)):
                if temp_board[row, col] == 0:
                    temp_board[row, col] = self.symbol
                    break
            state_str = str(temp_board.reshape(-1))
            value = self.value_function.get(state_str, 0.5)
            if value >= max_value:
                max_value = value
                best_col = col
        return best_col

    def update(self, board):
        self.positions.append(str(board.state.reshape(-1)))

    def reward(self, final_reward):
        for p in reversed(self.positions):
            if p not in self.value_function:
                self.value_function[p] = 0.5
            self.value_function[p] += self.alpha * (final_reward - self.value_function[p])
            final_reward = self.value_function[p]


#Clase Game:

In [14]:
class Game():
    def __init__(self, player1, player2):
        player1.symbol = 1
        player2.symbol = -1
        self.players = [player1, player2]
        self.board = Board()

    def selfplay(self, rounds=1000):
        wins = [0, 0]
        for i in tqdm(range(rounds)):
            self.board.reset()
            for p in self.players:
                p.reset()
            game_over = False
            turn = 0
            while not game_over:
                current_player = self.players[turn % 2]
                col = current_player.move(self.board)
                self.board.update(current_player.symbol, col)
                for p in self.players:
                    p.update(self.board)
                result = self.board.is_game_over()
                if result is not None:
                    game_over = True
                    break
                turn += 1
            self.reward()
            result = self.board.is_game_over()
            if result == self.players[0].symbol:
                wins[0] += 1
            elif result == self.players[1].symbol:
                wins[1] += 1
        return wins

    def reward(self):
        result = self.board.is_game_over()
        if result == 0:  # Empate
            for p in self.players:
                p.reward(0.5)
        else:
            for p in self.players:
                if p.symbol == result:
                    p.reward(1)
                else:
                    p.reward(0)


#Entrenamiento y análisis:

In [15]:
agent1 = Agent(prob_exp=0.1)
agent2 = Agent(prob_exp=0.1)

game = Game(agent1, agent2)
resultados = game.selfplay(10000)

print(f"Victorias del Agente 1: {resultados[0]}")
print(f"Victorias del Agente 2: {resultados[1]}")


100%|██████████| 10000/10000 [08:04<00:00, 20.64it/s]

Victorias del Agente 1: 5149
Victorias del Agente 2: 4215


Guardar la función de valor entrenada:

In [16]:
import pickle

with open("cuatro_en_raya_agente1.pickle", "wb") as handle:
    pickle.dump(agent1.value_function, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [18]:
import pandas as pd
funcion_de_valor = sorted(agent1.value_function.items(), key=lambda kv: kv[1], reverse=True)
tabla = pd.DataFrame({'estado': [x[0] for x in funcion_de_valor], 'valor': [x[1] for x in funcion_de_valor]})

tabla

,estado,valor
0,[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,0.96875
1,[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,0.93750
2,[ 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. ...,0.93750
3,[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,0.93750
4,[ 0. 0. 0. 0. 0. -1. -1. 0. 0. 0. 0. ...,0.87500
...,...,...
188948,[ 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. ...,0.12500
188949,[ 0. 0. 0. 0. 0. 1. -1. 0. 0. 0. 0. ...,0.12500
188950,[ 0. 0. 0. 0. 0. 1. -1. 0. 0. 0. 0. ...,0.12500
188951,[ 0. 0. 0. 0. 1. -1. -1. 0. 0. 0. 0. -...,0.12500


# Cargar desde drive funcion de valor del agente 1

In [26]:
import pickle
from google.colab import drive
drive.mount('/content/drive')

# Cargar función de valor entrenada
with open("/content/drive/MyDrive/Colab Notebooks/cuatro_en_raya_agente1.pickle", 'rb') as handle:
    value_function = pickle.load(handle)


Mounted at /content/drive


#Clase de agente que usa la función ya entrenada

In [27]:
class TrainedAgent():
    def __init__(self, symbol, value_function):
        self.symbol = symbol
        self.value_function = value_function

    def move(self, board):
        valid_moves = board.valid_moves()
        max_value = -1e9
        best_col = None
        for col in valid_moves:
            temp_board = board.state.copy()
            for row in reversed(range(ROWS)):
                if temp_board[row, col] == 0:
                    temp_board[row, col] = self.symbol
                    break
            state_str = str(temp_board.reshape(-1))
            value = self.value_function.get(state_str, 0.5)
            if value >= max_value:
                max_value = value
                best_col = col
        return best_col


#Jugar contra el agente por consola

In [28]:
def print_board(state):
    chars = {1: 'X', -1: 'O', 0: '.'}
    for row in state:
        print(' '.join([chars[int(cell)] for cell in row]))
    print()

def human_vs_agent():
    agent = TrainedAgent(symbol=1, value_function=value_function)
    board = Board()
    print("¡Bienvenido a Cuatro en Raya! Tú juegas como 'O' (colocas -1).")
    turn = 0
    while True:
        print_board(board.state)
        if turn % 2 == 0:
            # Turno del humano
            try:
                col = int(input("Tu movimiento (0-6): "))
                board.update(-1, col)
            except Exception as e:
                print("Movimiento inválido:", e)
                continue
        else:
            # Turno del agente
            col = agent.move(board)
            print(f"Agente juega en columna {col}")
            board.update(1, col)
        result = board.is_game_over()
        if result is not None:
            print_board(board.state)
            if result == 1:
                print("¡El agente ganó!")
            elif result == -1:
                print("¡Ganaste tú!")
            else:
                print("Empate.")
            break
        turn += 1

# Ejecutar el juego
human_vs_agent()


¡Bienvenido a Cuatro en Raya! Tú juegas como 'O' (colocas -1).
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .

Tu movimiento (0-6): 4
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . O . .

Agente juega en columna 3
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . X O . .

Tu movimiento (0-6): 4
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . O . .
. . . X O . .

Agente juega en columna 0
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . O . .
X . . X O . .

Tu movimiento (0-6): 5
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . O . .
X . . X O O .

Agente juega en columna 2
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . O . .
X . X X O O .

Tu movimiento (0-6): 1
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . O . .
X O X X O O .

Agente juega en columna 4
. . . . . . .
. . . . . . .
. . . . . . .
. . . . X . .
. . .